In [1]:
import numpy as np
import tensorflow as tf

from keras.preprocessing.sequence import pad_sequences
import pickle

Using TensorFlow backend.


In [2]:
import data_utils
import embedding_utils
import display_utils

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
np.random.seed(1001)
tf.set_random_seed(1001)

### Load data

In [5]:
VOCAB_SIZE  = 50000
#load dataset
with open('aux_files/dataset_%d.pkl' %VOCAB_SIZE, 'rb') as f:
    dataset = pickle.load(f)

In [6]:

#load conter-fitted word 
dist_mat = np.load('aux_files/dist_counter_%d.npy' %VOCAB_SIZE)
# Prevent returning 0 as most similar word because it is not part of the dictionary
dist_mat[0,:] = 100000
dist_mat[:,0] = 100000

skip_list = np.load('aux_files/missed_embeddings_counter_%d.npy' %VOCAB_SIZE)


In [7]:
doc_len = [len(dataset.val_X[i]) for i in 
           range(len(dataset.val_X))]

### Demonstrating how we find the most similar words

In [8]:
src_word = dataset.dict['好']
src_word
dataset.inv_full_dict[50000]

'伟'

In [9]:
for i in range(6230, 6232):
    src_word = i
    nearest, nearest_dist = embedding_utils.pick_most_similar_words(src_word, dist_mat,20, 0.5)
        
    print('Closest to `%s` are:' %(dataset.inv_dict[src_word]))
    for w_id, w_dist in zip(nearest, nearest_dist):
          print(' -- ', dataset.inv_dict[w_id], ' ', w_dist)

    print('----')

Closest to `国税局` are:
----
Closest to `翼` are:
 --  机翼   0.22788355864099952
 --  辅   0.22916311992500127
 --  翅膀   0.2738878111520009
 --  援   0.3341176668279999
 --  辅助   0.34568997297100035
 --  副   0.3563555582999989
 --  扶持   0.36099929984099965
 --  救援   0.3625574274400005
 --  支架   0.3828246505230002
 --  帮   0.390361236298
 --  援助   0.3941633762969998
 --  养   0.4035929243710008
 --  救   0.4049656322590014
 --  架   0.417898789983
 --  赞助   0.42399925209200195
 --  扶   0.42721877353100135
 --  资助   0.43061200651000053
 --  助益   0.4325585738910007
 --  声援   0.43391956708499957
 --  托   0.43443959343899996
----


### Loading the NLI model

In [10]:
from NLImodel.models import NLImodelClass


In [11]:
model = NLImodelClass(is_train=False)


Learning Rate:  0.0007
Model Loaded.


In [12]:
dataset.val_X[0], dataset.val_Y[0], dataset.val_Z[0], np.argmax(dataset.val_Z[0])

(array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
          328, 50000,    29,  2923,  1991,   236,   108, 50000,  2631,
           29,   106,  2348, 50000,  1099,    29,  2106,    15,    52,
          620,    62, 50000], dtype=int32),
 array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,    29,   900,   262,    15,    52,
         3915,  1670, 50000], dtype=int32),
 array([0., 0., 1.]),
 2)

In [13]:
np.argmax(model.predict(dataset.val_X[0], dataset.val_X[0])) 

1

## Try Attack

In [14]:
from attacks import GreedyAttack, GeneticAtack

### Main Attack 

In [15]:
pop_size = 32
n1 = 8
goog_lm = None

ga_atttack = GeneticAtack(model, model, model, dataset, dist_mat, 
                                  skip_list,
                                  goog_lm, max_iters=30, 
                                   pop_size=pop_size,
                                  
                                  n1 = n1,
                                  n2 = 4,
                                 use_lm = False, use_suffix=False)

In [16]:
len(dataset.val_Z)

5010

In [41]:
TEST_SIZE = len(dataset.val_Z)#1000
#use val data for attack
#test_idx = np.random.choice(len(dataset.val_Z), TEST_SIZE, replace=False)#随机index,总共1000个
test_idx = [x for x in range(len(dataset.val_Z))]
test_len = []
for i in range(TEST_SIZE):
    test_len.append(len(dataset.val_Y[test_idx[i]]))
print('Shortest sentence in our test set is %d words' %np.min(test_len))

test_list = []
orig_list = []
orig_list_no_max = []
orig_label_list = []
adv_list = []
dist_list = []

for i in range(TEST_SIZE):
    print('Index:',i)
    x_orig = [dataset.val_X[test_idx[i]], dataset.val_Y[test_idx[i]]]
    x_orig_no_max = [dataset.val_X_nomax[test_idx[i]], dataset.val_Y_nomax[test_idx[i]]]
    orig_label = np.argmax(dataset.val_Z[test_idx[i]])
    orig_preds=  model.predict(x_orig[0], x_orig[1])
    # print(orig_label, orig_preds, np.argmax(orig_preds))
    if np.argmax(orig_preds) != orig_label:
        print('skipping wrong classifed ..')
        print('--------------------------')
        continue
    x_len = np.sum(np.sign(x_orig[1]))
    '''
    if x_len >= 30:
        print('skipping too long input..')
        print('--------------------------')
        continue
    '''
    # if np.max(orig_preds) < 0.90:
    #    print('skipping low confidence .. \n-----\n')
    #    continue
    
    target_list = [0,1,2]
    target_list.remove(orig_label)
    print(orig_label, target_list)
    for target_label in target_list:
        print('****** ', len(test_list) + 1, ' ********')
        test_list.append(test_idx[i])
        orig_list.append(x_orig)
        orig_list_no_max.append(x_orig_no_max)
        #target_label = 1 if orig_label == 2 else 2
        orig_label_list.append(orig_label)
        x_adv = ga_atttack.attack( x_orig, target_label)
        adv_list.append(x_adv)
        if x_adv is None:
            print('%d failed' %(i+1))
            dist_list.append(100000)
        else:
            num_changes = np.sum(x_orig[1] != x_adv)
            print('%d - %d changed.' %(i+1, num_changes))
            dist_list.append(num_changes)
            # display_utils.visualize_attack(sess, model, dataset, x_orig, x_adv)

        print('--------------------------')
        #if (len(test_list)>= 50):
        #    break

Shortest sentence in our test set is 30 words
Index: 0
skipping wrong classifed ..
--------------------------
Index: 1
skipping wrong classifed ..
--------------------------
Index: 2
skipping wrong classifed ..
--------------------------
Index: 3
skipping wrong classifed ..
--------------------------
Index: 4
1 [0, 2]
******  1  ********
		 0  --  0.40092692
		 1  --  0.46689743
5 - 1 changed.
--------------------------
******  2  ********


/home/haejuu/aaai2019/haejuu/lib/python3.4/site-packages/ipykernel_launcher.py:55: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


		 0  --  0.33548075
		 1  --  0.3966511
5 - 1 changed.
--------------------------
Index: 5
skipping wrong classifed ..
--------------------------
Index: 6
2 [0, 1]
******  3  ********
		 0  --  0.13200133
		 1  --  0.2904192
		 2  --  0.38936827
		 3  --  0.5177411
7 - 1 changed.
--------------------------
******  4  ********
		 0  --  0.01855124
		 1  --  0.025707526
		 2  --  0.20765907
		 3  --  0.22423011
		 4  --  0.22423011
		 5  --  0.22423011
		 6  --  0.25113648
		 7  --  0.25113648
		 8  --  0.2584042
		 9  --  0.2584042
		 10  --  0.2584042
		 11  --  0.2584042
		 12  --  0.2584042
		 13  --  0.2584042
		 14  --  0.2584042
		 15  --  0.2584042
		 16  --  0.2584042
		 17  --  0.2584042
		 18  --  0.2584042
		 19  --  0.2584042
		 20  --  0.2584042
		 21  --  0.2584042
		 22  --  0.2584042
		 23  --  0.2584042
		 24  --  0.2584042
		 25  --  0.2584042
		 26  --  0.2584042
		 27  --  0.2584042
		 28  --  0.2584042
		 29  --  0.2584042
7 failed
--------------------------
Index:

		 10  --  0.124292605
		 11  --  0.124292605
		 12  --  0.12593691
		 13  --  0.12593691
		 14  --  0.12593691
		 15  --  0.12593691
		 16  --  0.12593691
		 17  --  0.12593691
		 18  --  0.12593691
		 19  --  0.12593691
		 20  --  0.12593691
		 21  --  0.12593691
		 22  --  0.12593691
		 23  --  0.12593691
		 24  --  0.12593691
		 25  --  0.12593691
		 26  --  0.12593691
		 27  --  0.12593691
		 28  --  0.12593691
		 29  --  0.12593691
31 failed
--------------------------
Index: 31
1 [0, 2]
******  41  ********
		 0  --  0.41226655
		 1  --  0.70717084
32 - 1 changed.
--------------------------
******  42  ********
		 0  --  0.2987586
		 1  --  0.7904016
32 - 1 changed.
--------------------------
Index: 32
2 [0, 1]
******  43  ********
		 0  --  0.25835398
		 1  --  0.50712585
33 - 1 changed.
--------------------------
******  44  ********
		 0  --  0.41718203
33 - 1 changed.
--------------------------
Index: 33
skipping wrong classifed ..
--------------------------
Index: 34
0 [1, 2

		 0  --  0.2002578
		 1  --  0.22012137
		 2  --  0.25214118
		 3  --  0.25214118
		 4  --  0.25214112
		 5  --  0.25214112
		 6  --  0.25214112
		 7  --  0.25214112
		 8  --  0.25214112
		 9  --  0.25214112
		 10  --  0.25214112
		 11  --  0.25214112
		 12  --  0.25214112
		 13  --  0.25214112
		 14  --  0.25214112
		 15  --  0.25214112
		 16  --  0.25214112
		 17  --  0.25214112
		 18  --  0.25214112
		 19  --  0.25214112
		 20  --  0.25214112
		 21  --  0.25214112
		 22  --  0.25214112
		 23  --  0.25214112
		 24  --  0.25214112
		 25  --  0.25214112
		 26  --  0.25214112
		 27  --  0.25214112
		 28  --  0.25214112
		 29  --  0.25214112
59 failed
--------------------------
******  70  ********
		 0  --  0.21904036
		 1  --  0.24312499
		 2  --  0.25659013
		 3  --  0.25659013
		 4  --  0.25861815
		 5  --  0.25861815
		 6  --  0.25861815
		 7  --  0.25861815
		 8  --  0.25861815
		 9  --  0.25861812
		 10  --  0.25861812
		 11  --  0.25861812
		 12  --  0.25861812
		 13  --  0.2586

...

		 3  --  0.27055696
		 4  --  0.27215785
		 5  --  0.27215785
		 6  --  0.29887652
		 7  --  0.31291184
		 8  --  0.31291184
		 9  --  0.31291184
		 10  --  0.31291184
		 11  --  0.31291184
		 12  --  0.31291184
		 13  --  0.31291184
		 14  --  0.31291184
		 15  --  0.31291184
		 16  --  0.31291184
		 17  --  0.31291184
		 18  --  0.31291184
		 19  --  0.31291184
		 20  --  0.31291184
		 21  --  0.31291184
		 22  --  0.31291184
		 23  --  0.31291184
		 24  --  0.31291184
		 25  --  0.31291184
		 26  --  0.31291184
		 27  --  0.31291184
		 28  --  0.31291184
		 29  --  0.31291184
1278 failed
--------------------------
******  1600  ********
		 0  --  0.43168518
1278 - 1 changed.
--------------------------
Index: 1278
skipping wrong classifed ..
--------------------------
Index: 1279
2 [0, 1]
******  1601  ********
		 0  --  0.44961852
1280 - 1 changed.
--------------------------
******  1602  ********
		 0  --  0.4083751
1280 - 1 changed.
--------------------------
Index: 1280
skipping

		 3  --  0.40816754
1317 - 1 changed.
--------------------------
Index: 1317
2 [0, 1]
******  1639  ********
		 0  --  0.3726425
		 1  --  0.6124978
1318 - 1 changed.
--------------------------
******  1640  ********
		 0  --  0.028358534
		 1  --  0.036092997
		 2  --  0.03610883
		 3  --  0.03610883
		 4  --  0.037991397
		 5  --  0.037991397
		 6  --  0.037991397
		 7  --  0.037991397
		 8  --  0.037991397
		 9  --  0.037991397
		 10  --  0.037991397
		 11  --  0.037991397
		 12  --  0.037991397
		 13  --  0.037991397
		 14  --  0.037991397
		 15  --  0.037991382
		 16  --  0.037991382
		 17  --  0.037991382
		 18  --  0.037991382
		 19  --  0.037991382
		 20  --  0.037991382
		 21  --  0.037991382
		 22  --  0.037991382
		 23  --  0.037991382
		 24  --  0.037991382
		 25  --  0.037991382
		 26  --  0.037991382
		 27  --  0.037991382
		 28  --  0.037991382
		 29  --  0.037991382
1318 failed
--------------------------
Index: 1318
skipping wrong classifed ..
-------------------------

		 0  --  0.36216936
		 1  --  0.51345545
1344 - 1 changed.
--------------------------
******  1664  ********
		 0  --  0.30366412
		 1  --  0.39974344
1344 - 1 changed.
--------------------------
Index: 1344
skipping wrong classifed ..
--------------------------
Index: 1345
0 [1, 2]
******  1665  ********
		 0  --  0.34601536
		 1  --  0.42405483
1346 - 1 changed.
--------------------------
******  1666  ********
		 0  --  0.3625722
		 1  --  0.45073482
1346 - 1 changed.
--------------------------
Index: 1346
skipping wrong classifed ..
--------------------------
Index: 1347
skipping wrong classifed ..
--------------------------
Index: 1348
skipping wrong classifed ..
--------------------------
Index: 1349
skipping wrong classifed ..
--------------------------
Index: 1350
skipping wrong classifed ..
--------------------------
Index: 1351
2 [0, 1]
******  1667  ********
		 0  --  0.45782948
1352 - 1 changed.
--------------------------
******  1668  ********
		 0  --  0.3514944
		 1  --

		 4  --  0.3745975
		 5  --  0.37459752
		 6  --  0.37459752
		 7  --  0.37459752
		 8  --  0.37459752
		 9  --  0.37459752
		 10  --  0.37459752
		 11  --  0.37459752
		 12  --  0.37459752
		 13  --  0.37459752
		 14  --  0.37459752
		 15  --  0.37459752
		 16  --  0.37459752
		 17  --  0.37459752
		 18  --  0.37459752
		 19  --  0.37459752
		 20  --  0.37459752
		 21  --  0.37459752
		 22  --  0.37459752
		 23  --  0.37459752
		 24  --  0.37459752
		 25  --  0.37459752
		 26  --  0.37459752
		 27  --  0.37459752
		 28  --  0.37459752
		 29  --  0.37459752
1385 failed
--------------------------
Index: 1385
1 [0, 2]
******  1713  ********
		 0  --  0.41237602
1386 - 1 changed.
--------------------------
******  1714  ********
		 0  --  0.6024425
1386 - 1 changed.
--------------------------
Index: 1386
skipping wrong classifed ..
--------------------------
Index: 1387
1 [0, 2]
******  1715  ********
		 0  --  0.35354853
1388 - 1 changed.
--------------------------
******  1716  *******

ValueError: Error when checking : expected input_2 to have shape (None, 30) but got array with shape (0, 1)

### Visualize some output results

In [32]:
print('Median number of modifications ' , np.median([x for x in dist_list if x != 100000]))

Median number of modifications  1.0


In [33]:
orig_len = [np.sum(np.sign(x)) for x in orig_list[1]]
ratio = [dist_list[i]/orig_len[i] for i in range(len(orig_list[1]))]

In [34]:
ratio_success = [x for x in ratio if x < 1.0]
np.mean(ratio_success)

0.04444444444444444

In [42]:
#display_utils recoverd max vocabulary limitation, all words with 50000 index were recovered to original index. So sentences showed below may be different from what NLI model see.
for visual_idx in range(len(adv_list)):
    print(visual_idx+1)
    display_utils.visualize_attack(model, dataset, orig_list[visual_idx], adv_list[visual_idx], orig_list_no_max[visual_idx],)

1
Original Prediction = entailment. (Confidence = 51.89) 
Premise


Hypothesis


---------  After attack -------------
New Prediction = neutral. (Confidence = 46.69) 


2
Original Prediction = entailment. (Confidence = 51.89) 
Premise


Hypothesis


...

Hypothesis


---------  After attack -------------
New Prediction = neutral. (Confidence = 47.16) 


1758
Original Prediction = contradiction. (Confidence = 41.67) 
Premise


Hypothesis


---------  After attack -------------
New Prediction = entailment. (Confidence = 43.53) 
